In [ ]:
import os
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split

# ¿Que dicen los LLM de nuestros datos?

Primero importaremos los datos y le aplicaremos la función de rating

In [ ]:
# leemos el csv
df = pd.read_csv("threads.csv")
df = df.drop(["source", "review_date"], axis=1)
# change the column names to match the HuggingFace dataset format
df = df.rename(columns={"review": "text", "rating": "label"})

# función para convertir el rating a palabras
def ratingTransform(rating):
    if rating <= 2:
        return 0
    elif rating <= 4:
        return 1
    else:
        return 2

# Transform the ratings to positive, neutral, and negative
df["label"] = df["label"].apply(ratingTransform)

train, test = train_test_split(df, test_size=0.33)

# guardamos los datos en un csv
train.to_csv("train.csv", index=False)
test.to_csv("test.csv", index=False)

# cargamos los datos en un dataset de huggingface
dataset = load_dataset("csv", data_files={"train": "train.csv", "test": "test.csv"})

## Usando Tokenizers

Los tokenizers son el proceso de convertir una secuencia de texto en una secuencia de tokens (números que hacen referencia a palabras).

In [ ]:
# Entrenemos un modelo DistilBERT
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenizamos los datos
def tokenize_data(example):
    return tokenizer(example['review_description'], padding='max_length')

# tokenizamos los datos
dataset = dataset.map(tokenize_data, batched=True)

## Cargamos el modelo DistilBERT

DistilBERT es un modelo de BERT que es más pequeño y rápido, pero que mantiene una precisión similar a la de BERT.

In [ ]:
from transformers import AutoModelForSequenceClassification

# Cargamos un modelo pre-entrenado
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

batch_size = 8
number_of_epochs = 7
logging_steps = len(dataset['train']) // batch_size
steps = (len(dataset['train']) // batch_size) * number_of_epochs
warmup_steps = int(0.2*steps)

from transformers import TrainingArguments

training_args = TrainingArguments(
                                  num_train_epochs=number_of_epochs, 
                                  load_best_model_at_end=True,
                                  evaluation_strategy='steps', 
                                  save_strategy='steps',
                                  learning_rate=2e-5,
                                  logging_steps=logging_steps,
                                  warmup_steps= warmup_steps,
                                  save_steps=1000,
                                  eval_steps=500,
                                  output_dir="fine-tuned-distilbert-base-uncased"
                                  )

# shuffle the dataset
train_dataset = dataset['train'].shuffle(seed=10) 
eval_dataset = dataset['test'].shuffle(seed=10)

# Fine-tuning de DistilBERT

El fine-tuning es un proceso de entrenamiento en el que se usa un modelo pre-entrenado y se ajusta para que se adapte a los datos específicos del problema. En este caso, usaremos el modelo pre-entrenado de DistilBERT y lo ajustaremos para que se adapte a nuestros datos de entrenamiento.

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
                )

trainer.train()

# Evaluación del modelo

Ahora que hemos entrenado nuestro modelo, podemos evaluarlo en el conjunto de datos de prueba, para ver cómo se desempeña en datos que nunca ha visto antes.

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error
from datasets import load_metric


def compute_metrics(eval_pred):
    # load the metrics to use
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # calculate the mertic using the predicted and true value 
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)
    f1 = load_f1.compute(predictions=predictions, references=labels, average="weighted")
    return {"accuracy": accuracy, "f1score": f1}

trainer.evaluate()

# Interpretación del modelo

Ahora veremos que palabras son las que más influyen en la predicción del modelo.

In [ ]:
# Interpretabilidad del modelo

from transformers_interpret import SequenceClassificationExplainer

# load the explainer
explainer = SequenceClassificationExplainer(
    model,
    tokenizer
)

# explain the first sample in the test set
idx = 0
explanation = explainer(dataset['test'][idx]['input_ids'])

# print the results
print("The word importance is: ", explanation.word_importances)
print("The tokens are: ", explanation.words)
print("The positive and negative contributions are: ", explanation.contributions)
print("The attribution is: ", explanation.attributions)
print("The visualization is: ")
explanation.visualize()